In questo notebook andremo a preprocessare tutti i file JSON per creare dei dataset di TRAIN, VALIDATION e TEST utili e pronti per il progetto

Questa funzione aggiunge l'attributo semantic_category del file metadata nel file train, validation o test

In [4]:
import json
def aggiungi_categorie_semantiche(file1_path, file2_path, output_path):
    # Carica il contenuto del primo file JSON (descrizione degli outfit)
    with open(file1_path, 'r') as file1:
        data1 = json.load(file1)

    # Carica il contenuto del secondo file JSON
    with open(file2_path, 'r') as file2:
        data2 = json.load(file2)

    # Crea un dizionario di mapping item_id -> semantic_category dal secondo file
    id_to_category = {item_id: item["semantic_category"] for item_id, item in data2.items()}

    # Itera attraverso i dizionari nella lista data1
    for item_dict in data1:
        # Accedi all'array "items" all'interno del dizionario corrente
        items_list = item_dict["items"]

        # Aggiungi le semantic_category agli oggetti nell'array "items"
        for item in items_list:
            item_id = item["item_id"]
            semantic_category = id_to_category.get(item_id)
            if semantic_category:
                item["semantic_category"] = semantic_category

    # Salva il nuovo contenuto nel primo file JSON aggiornato
    with open(output_path, 'w') as file1_updated:
        json.dump(data1, file1_updated, indent=4)


Richiamiamo la funzione per i 3 set (training, validation e test) 
Ci saranno 2 tipologie di "gruppi" di set, 1 gruppo uguale per la categoria bottom e shoes, e 1 gruppo per gli accessori che necessitano dell'attributo category_id per lavorare

In [5]:
# Utilizzo della funzione per il train set:
file1_path = 'C:/Users\marce\Desktop\polyvore_outfits/disjoint/train.json'
file2_path = 'C:/Users\marce\Desktop\polyvore_outfits/polyvore_item_metadata.json'
output_path = 'Dataset/train_set_bs.json'
aggiungi_categorie_semantiche(file1_path, file2_path, output_path)

# Lavoriamo nello stesso modo per preprocessare il validation set
file1_path = 'C:/Users/marce/Desktop/polyvore_outfits/disjoint/valid.json'
file2_path = 'C:/Users/marce/Desktop/polyvore_outfits/polyvore_item_metadata.json'
output_path = 'Dataset/valid_set_bs.json'
aggiungi_categorie_semantiche(file1_path, file2_path, output_path)

# Lavoriamo nello stesso modo per preprocessare il test set
file1_path = 'C:/Users/marce/Desktop/polyvore_outfits/disjoint/test.json'
file2_path = 'C:/Users/marce/Desktop/polyvore_outfits/polyvore_item_metadata.json'
output_path = 'Dataset/test_set_bs.json'
aggiungi_categorie_semantiche(file1_path, file2_path, output_path)

Eliminiamo gli outfit che hanno due items con la stessa categoria

In [6]:
import json

def filtra_dati_e_salva(input_file_path, output_file_path):
    # Carica il file JSON
    with open(input_file_path, 'r') as json_file:
        json_data = json.load(json_file)

    # Nuova struttura dati con elementi filtrati
    filtered_data = []

    # Itera attraverso gli elementi nel JSON
    for entry in json_data:
        new_entry = {
            "items": [],
            "set_id": entry["set_id"]
        }

        # Dizionario per tenere traccia delle categorie semantiche per ogni "item"
        item_semantic_categories = {}

        for item in entry["items"]:
            category = item["semantic_category"]
            if category not in item_semantic_categories:
                item_semantic_categories[category] = True
                new_entry["items"].append(item)

        filtered_data.append(new_entry)

    # Sovrascrive il file JSON originale con i dati filtrati
    with open(output_file_path, 'w') as json_file:
        json.dump(filtered_data, json_file, indent=4)


Eseguiamo la funzione sui 3 set 

In [7]:
# Utilizzo della funzione per il train set:
input_file_path = 'Dataset/train_set_bs.json'
output_file_path = 'Dataset/train_set_bs.json'
filtra_dati_e_salva(input_file_path, output_file_path)

# Facciamo la stessa cosa per il validation set
output_path = 'Dataset/valid_set_bs.json'
filtra_dati_e_salva(output_path,'Dataset/valid_set_bs.json')

#Idem per il test set 
output_path = 'Dataset/test_set_bs.json'
filtra_dati_e_salva(output_path,'Dataset/test_set_bs.json')


Questa funzione elimina gli outfit con meno di 4 elementi al proprio interno

In [8]:
import json

def outfit_con_4_elementi(input_file_path, output_file_path, min_item_count=4):
    # Carica il file JSON
    with open(input_file_path, 'r') as json_file:
        json_data = json.load(json_file)

    # Nuova struttura dati con elementi filtrati
    filtered_data = []

    # Itera attraverso gli elementi nel JSON
    for entry in json_data:
        if len(entry["items"]) >= min_item_count:
            filtered_items = [item for item in entry["items"] if item]
            if len(filtered_items) >= min_item_count:
                new_entry = {
                    "items": filtered_items,
                    "set_id": entry["set_id"]
                }
                filtered_data.append(new_entry)

    # Sovrascrive il file JSON originale con i dati filtrati
    with open(output_file_path, 'w') as json_file:
        json.dump(filtered_data, json_file, indent=4)


Eseguiamo la funzione sui 3 set (che verranno utilizzati da Bottoms e Shoes)

In [9]:
# Utilizzo della funzione per il training set:
input_file_path = 'Dataset/train_set_bs.json'
output_file_path = 'Dataset/train_set_bs.json'
outfit_con_4_elementi(input_file_path, output_file_path)

# Utilizzo della funzione per il validation set:
input_file_path = 'Dataset/valid_set_bs.json'
output_file_path = 'Dataset/valid_set_bs.json'
outfit_con_4_elementi(input_file_path, output_file_path)

# Utilizzo della funzione per il validation set:
input_file_path = 'Dataset/test_set_bs.json'
output_file_path = 'Dataset/test_set_bs.json'
outfit_con_4_elementi(input_file_path, output_file_path)

Questa funzione è un "upgrade" della funzione "aggiungi_categorie_semantiche". La novità è che aggiunge nel file train/valid e test .json l'attributo "category_id" corrispondente per ogni categoria semantica

In [10]:
import json
def aggiungi_categorie_semantiche_acc(file1_path, file2_path, output_path):
    # Carica il contenuto del primo file JSON (descrizione degli outfit)
    with open(file1_path, 'r') as file1:
        data1 = json.load(file1)

    # Carica il contenuto del secondo file JSON
    with open(file2_path, 'r') as file2:
        data2 = json.load(file2)

    # Crea un dizionario di mapping item_id -> {"semantic_category", "category_id"} dal secondo file
    id_to_info = {item_id: {"semantic_category": item["semantic_category"], "category_id": item.get("category_id")} for item_id, item in data2.items()}

    # Itera attraverso i dizionari nella lista data1
    for item_dict in data1:
        # Accedi all'array "items" all'interno del dizionario corrente
        items_list = item_dict["items"]

        # Aggiungi le semantic_category e category_id agli oggetti nell'array "items"
        for item in items_list:
            item_id = item["item_id"]
            info = id_to_info.get(item_id)
            if info:
                item["semantic_category"] = info["semantic_category"]
                item["category_id"] = info["category_id"]

    # Salva il nuovo contenuto nel primo file JSON aggiornato
    with open(output_path, 'w') as file1_updated:
        json.dump(data1, file1_updated, indent=4)


Lavoro col file train della cartella nondisjoint perchè alcune category_id come per esempio 52, che rappresenta le cinture, non è presente all'interno del file train della cartella disjoint

In [11]:
# Utilizzo della funzione per il training set di accessories:
file1_path = 'C:/Users/marce/Desktop/polyvore_outfits/nondisjoint/train.json'
file2_path = 'C:/Users/marce/Desktop/polyvore_outfits/polyvore_item_metadata.json'
output_path = 'Dataset/train_set_acc.json'
aggiungi_categorie_semantiche_acc(file1_path, file2_path, output_path)

# Utilizzo della funzione per il validation set di accessories:
file1_path = 'C:/Users/marce/Desktop/polyvore_outfits/nondisjoint/valid.json'
file2_path = 'C:/Users/marce/Desktop/polyvore_outfits/polyvore_item_metadata.json'
output_path = 'Dataset/valid_set_acc.json'
aggiungi_categorie_semantiche_acc(file1_path, file2_path, output_path)

# Utilizzo della funzione per il test set di accessories:
file1_path = 'C:/Users/marce/Desktop/polyvore_outfits/nondisjoint/test.json'
file2_path = 'C:/Users/marce/Desktop/polyvore_outfits/polyvore_item_metadata.json'
output_path = 'Dataset/test_set_acc.json'
aggiungi_categorie_semantiche_acc(file1_path, file2_path, output_path)

Dal file csv categories.csv recupero solo le righe che hanno come semantic_categories accessories

In [12]:
import csv

# Apri il file CSV di input in modalità di lettura
with open('C:/Users/marce/Desktop/polyvore_outfits/categories.csv', 'r') as input_file:
    reader = csv.reader(input_file)
    
    # Crea una lista per le righe che soddisfano il criterio
    selected_rows = [row for row in reader if row[-1] == 'accessories']

# Nome del file CSV di output
output_file = 'categories_accessories.csv'

# Scrivi le righe selezionate nel nuovo file CSV
with open(output_file, 'w', newline='') as out_file:
    writer = csv.writer(out_file)
    writer.writerows(selected_rows)

print(f"Le righe con 'accessories' sono state scritte su {output_file}")

Le righe con 'accessories' sono state scritte su categories_accessories.csv


Stampo a video la tabella csv delle tipologie di accessories

In [13]:
import pandas as pd
df = pd.read_csv(output_file)

# Stampa la tabella CSV a video
print(df)

      53                       gloves  accessories
0     56                          tie  accessories
1     58                     headband  accessories
2     59                     umbrella  accessories
3     68                    stockings  accessories
4     69                        socks  accessories
5    231                        pouch  accessories
6    251                      hosiery  accessories
7    269                   suspenders  accessories
8    270           activity wristband  accessories
9    299             male accessories  accessories
10   300                    male belt  accessories
11   301                  male gloves  accessories
12   302                 male scarves  accessories
13   306              male suspenders  accessories
14  4460                   male socks  accessories
15  4463                       wallet  accessories
16  4467                    key chain  accessories
17  4468                   money clip  accessories
18  4470                       

Ora andiamo a sostituire il valore intero dell'attributo "category_id" con il secondo valore della riga corrispondente nel file csv categories.csv

In [14]:
import csv
import json

def replace_category_ids(input_json_path, output_json_path):
    # Carica il file JSON
    with open(input_json_path, 'r') as json_file:
        data = json.load(json_file)

    # Crea un dizionario per mappare i valori di "category_id" con i valori dalla colonna CSV
    category_mapping = {}
    with open('categories_accessories.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            category_id = row[0]
            category_value = row[1]
            category_mapping[category_id] = category_value

    # Funzione per sostituire il valore di "category_id"
    def replace_category_id(item):
        category_id = item.get("category_id")
        if category_id == "52":
            item["category_id"] = "belt"
        elif category_id in category_mapping:
            item["category_id"] = category_mapping[category_id]

    # Applica la sostituzione ai dati JSON
    for entry in data:
        for item in entry["items"]:
            replace_category_id(item)

    # Salva i dati modificati in un nuovo file JSON
    with open(output_json_path, 'w') as new_json_file:
        json.dump(data, new_json_file, indent=4)

    print("Sostituzione completata.")

In [15]:
# utilizzo della funzione per il training set di accessories
replace_category_ids('Dataset/train_set_acc.json', 'Dataset/train_set_acc.json')

# utilizzo della funzione per il validation set di accessories
replace_category_ids('Dataset/valid_set_acc.json', 'Dataset/valid_set_acc.json')

# utilizzo della funzione per il test set di accessories
replace_category_ids('Dataset/test_set_acc.json', 'Dataset/test_set_acc.json')


Sostituzione completata.
Sostituzione completata.
Sostituzione completata.


Ora possiamo effettuare le stesse operazioni di preprocessing dell'altro gruppo di dataset

In [16]:
# Per il training set di accessories
filtra_dati_e_salva('Dataset/train_set_acc.json','Dataset/train_set_acc.json')
# Per il validation set di accessories
filtra_dati_e_salva('Dataset/valid_set_acc.json','Dataset/valid_set_acc.json')
# Per il test set di accessories
filtra_dati_e_salva('Dataset/test_set_acc.json', 'Dataset/test_set_acc.json')

In [17]:
# Per il training set di accessories
outfit_con_4_elementi('Dataset/train_set_acc.json','Dataset/train_set_acc.json')
# Per il validation set di accessories
outfit_con_4_elementi('Dataset/valid_set_acc.json','Dataset/valid_set_acc.json')
# Per il test set di accessories
outfit_con_4_elementi('Dataset/test_set_acc.json', 'Dataset/test_set_acc.json')

Determino la funzione della creazione dell'embedding partendo da un'immagine di un top tramite Residual NN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model

def get_image_embedding(image_id):
    # Carico il modello RESNet50 pre-allenato senza l'ultimo classification layer
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

    # Aggiungo uno strato di Global Average Pooling per ridurre le dimensioni dell'output
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Aggiungo un FC layer con 128 unità per ottenere un'embedding di dimensioni (1, 128)
    embedding_layer = tf.keras.layers.Dense(128)(x)

    # Costruisco il nuovo modello
    model = Model(inputs=base_model.input, outputs=embedding_layer)
    
    # Carica l'immagine
    img_path = f'C:/Users/marce/Desktop/polyvore_outfits/images/{image_id}.jpg'  # Utilizzo l'image_id passato come argomento
    img = image.load_img(img_path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    # Mostra l'immagine
    plt.imshow(img)
    plt.axis('off')  # Rimuovi gli assi
    plt.show()
    
    # Calcola l'embedding
    embedding = model.predict(x)

    return embedding

# Utilizzo la funzione per ottenere un'embedding di dimensioni (1, 128) di un'immagine specifica
#Abbiamo scelto questa immagine del top scorrendo le varie immagini nella cartella images
image_id = 178260114

embedding = get_image_embedding(image_id)

# Visualizza l'embedding
print("Embedding dell'immagine tops: ", embedding)

# Visualizza le dimensioni
print("Dimensioni embedding: ", embedding.shape)

# Salva l'embedding permanentemente in un file NumPy
np.save('embedding.npy', embedding)